In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.contrib.tensorboard.plugins import projector

In [3]:
#载入数据集
mnist=input_data.read_data_sets("MNIST_data/",one_hot=True)

#运行次数
max_steps=1001


#图片数量
image_num=3000

#文件路径 （程序所在路径）
DIR="/Users/yunhuarong/"

#定义会话
sess=tf.Session()

#载入图片
embedding=tf.Variable(tf.stack(mnist.test.images[:image_num]),trainable=False,name='embedding')

#参数概要
def variable_summaries(var):
    with tf.name_scope('summaires'):
        mean=tf.reduce_mean(var)
        #平均值
        tf.summary.scalar('mean',mean)
        with tf.name_scope('stddev'):
            stddev=tf.sqrt(tf.reduce_mean(tf.square(var-mean)))
        tf.summary.scalar('stddev',stddev)
        tf.summary.scalar('max',tf.reduce_max(var))
        tf.summary.scalar('min',tf.reduce_min(var))
        tf.summary.histogram('histogram',var)
        
#命名空间
with tf.name_scope('input'):
    #None 表示第一个维度为任意长度
    x=tf.placeholder(tf.float32,[None,784],name='x-input')
    #正确的标签
    y=tf.placeholder(tf.float32,[None,10],name='y-input')
    
#显示图片
with tf.name_scope('input_reshape'):
        image_shaped_input=tf.reshape(x,[-1,28,28,1])
        tf.summary.image('input',image_shaped_input,10)
        
with tf.name_scope('layer'):
    
    #创建简单神经网络
    with tf.name_scope('weights'):
        W=tf.Variable(tf.zeros([784,10]),name='W')
        variable_summaries(W)
    with tf.name_scope('biases'):    
        b=tf.Variable(tf.zeros([10]),name='b')
        variable_summaries(b)
    with tf.name_scope('wx_plus_b'):    
        wx_plus_b=tf.matmul(x,W)+b
    with tf.name_scope('soft_max'):
        prediction=tf.nn.softmax(wx_plus_b)  

with tf.name_scope('loss'):
    #交叉墒训练
    loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=prediction))
    tf.summary.scalar('loss',loss)

with tf.name_scope('train'):
#使用梯度下降法
    train_step=tf.train.GradientDescentOptimizer(0.2).minimize(loss)

#初始化变量
init=tf.global_variables_initializer()

with tf.name_scope('accuracy'):
    with tf.name_scope('correct_prediction'):
        #结果存放在布尔型列表中  定义准确率 比较两个agrmax大小，相同就是TRUE 不同就是FALSE。 argmax求最大值所在位置
        correct_prediction=tf.equal(tf.argmax(y,1),tf.argmax(prediction,1))

    with tf.name_scope('accuracy'):
        #求准确率 (tf.cast()将布尔型数据转化成数值型 然后计算准确率)
        accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
        tf.summary.scalar('accuracy',accuracy)
        

        
#产生metadata文件
#检查指定路径是否有此文件，如果有删除
if tf.gfile.Exists(DIR+'projector/projector/metadata.tsv'):
    tf.gfile.DeleteRecursively(DIR+'projector/projector/metadata.tsv')
with open(DIR+'/projector/projector/metadata.tsv','w')as f:
    #查找1所在位置，提取位置就是标签
    labels=sess.run(tf.argmax(mnist.test.labels[:],1))
        #循环
    for i in range(image_num):
            #每次写入后隔开一行
            f.write(str(labels[i])+'\n')
            
#合并所有summaries
merged=tf.summary.merge_all()

#官网visualization例子
projector_writer=tf.summary.FileWriter(DIR+'projector/projector',sess.graph)
saver=tf.train.Saver()
config=projector.ProjectorConfig()
embed=config.embeddings.add()
embed.tensor_name=embedding.name
embed.metadata_path=DIR +'projector/projector/metadata.tsv'
embed.sprite.image_path=DIR +'projector/data/mnist_10k_sprite.png'
embed.sprite.single_image_dim.extend([28,28])
projector.visualize_embeddings(projector_writer,config)

with tf.Session() as sess:
    sess.run(init)
    writer=tf.summary.FileWriter('logs/tensorboard_test/10/',sess.graph)
    for i in range(max_steps):
        batch_xs,batch_ys=mnist.train.next_batch(100)
        run_options=tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
        run_metadata=tf.RunMetadata() 
        summary,_=sess.run([merged,train_step],feed_dict={x:batch_xs,y:batch_ys},options=run_options,run_metadata=run_metadata)
        projector_writer.add_run_metadata(run_metadata,'step%03d'% i)
        projector_writer.add_summary(summary,i)
    
        if i%100 == 0:
                acc=sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels})
                print("Iter"+str(i)+", Testing accuracy="+str(acc))
            
    saver.save(sess,DIR +'projector/projector/a_model.ckpt',global_step=max_steps)
    projector_writer.close()
sess.close()

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Iter0, Testing accuracy=0.1053
Iter100, Testing accuracy=0.7515
Iter200, Testing accuracy=0.8007
Iter300, Testing accuracy=0.8124
Iter400, Testing accuracy=0.8186
Iter500, Testing accuracy=0.8235
Iter600, Testing accuracy=0.8576
Iter700, Testing accuracy=0.8737
Iter800, Testing accuracy=0.8829
Iter900, Testing accuracy=0.8888
Iter1000, Testing accuracy=0.893
